<font size="5" color='red'> <b> This script does not include the correction for the machine if your LVDTs are too far from the sample <b> </font>

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import linregress
from scipy.signal import butter,filtfilt, find_peaks

In [ ]:
offset = 0.1                                                                                    # It is the offset in the slope equation  y=ax+b to change the position of your line. Normally no more than 0.5

In [ ]:
df_exp_PB = pd.read_csv('File.csv')

# Filter the data, and plot both the original and filtered signals.
stress = df_exp_PB['Stress [MPa]']                                                              # Stress from csv
strain = df_exp_PB['Strain Ax [-]']                                                             # Strain from csv


# Maximum stress point
y = stress.max()                                                                                # Find maximum stress of the 
max_index = stress.argmax()
# print(max_index)                                                                              # 2357 is the index of the maximum value
x = strain[max_index]                                                                           # Apply the maximum index of the stress to the strain


# Filter
index_min = stress[stress > 0.3].index[0]                                                       # This filters the data from values of stress more than 0.3 if you are not interested in the messy part at the beginning
df_exp_PB_short = df_exp_PB.iloc[index_min : max_index + 100]                                   # If you recorded all the test included the return, this will filter that return. You can change the 100 if you want to show more data from the max stress forward

stress1 = df_exp_PB_short['Stress [MPa]']                                                        
strain1 = df_exp_PB_short['Strain Ax [-]']

stress2 = stress1.to_numpy()
strain2 = strain1.to_numpy()

In [ ]:
## Fracture strength: point of strain where the material physically separates. 
# At this point, the strain reaches its maximum value and the material actually fractures, even though the corresponding stress may be less than the ultimate strength at this point
fract_strain =  strain2.max()
max_index_fract = strain2.argmax()
fract_stress = stress2[max_index_fract]
# fract_stress = np.interp(fract_strain, strain1, stress1)
FractS = f'The fracture strength is {round(fract_stress,1)} MPa'
print(fract_strain, fract_stress)

In [ ]:
###ONLY FOR LINEAR REGRESSION
# Plot the young modulus slope
stress_range = stress2[(stress2 >= 'Number from y-axes') & (stress2 <= 'Number from y-axes')]                               # stress filtered for values more than 15 less 33
strain_range = strain2[(stress2 >= 'Number from y-axes') & (stress2 <= 'Number from y-axes')]                               # strain filtered for values more than 15 less 33

## Linear regression to fit the linear slope corresponding to the Young's
linear_regression_output = linregress(strain_range, stress_range)
print(linear_regression_output)
E = linear_regression_output[0]                                                         # Elastic modulus or young's modulus
b = linear_regression_output[1]
print(f'The elastic modulus is {round(E,1)} MPa')

ElasticM = f'The elastic modulus is {round(E/1000,1)} GPa'                              # Text to print in plot

stress_offset = E*(strain2) + b - offset                                                # Slope y=ax+b. Line to plot





## Find yield strength
for i in range(2, len(strain2)):
    if stress_offset[i] >= stress2[i]:
        sx1 = strain2[i-1]
        sy1 = stress2[i-1]
        sy2 = stress2[i]
        sx2 = strain2[i]

        ox1 = strain2[i-1]
        oy1 = stress_offset[i-1]
        ox2 = stress2[i]
        oy2 = stress_offset[i]
        break

# print(sx1, sy1, sx2, sy2, ox1, oy1, ox2, oy2)

x1 = ox1
y1 = oy1
x2 = ox2
y2 = oy2

x3 = sx1
y3 = sy1
x4 = sx2
y4 = sy2

YS = ( (x1*y2 - y1*x2)*(y3-y4) - (y1-y2)*(x3*y4-y3*x4) ) / ( ( (x1-x2)*(y3-y4))- ( (y1-y2)*(x3-x4)))
print(YS)


# If you want to create an horizontal line to visualize the intersection
xi = (YS-b) / (E)
yi = YS

print('(xi,yi)',xi,yi)

In [ ]:
### PLOTTING
## Plot all the data as interactive plot
ax = px.line(df_exp_PB, x = 'Strain Ax [-]', y = 'Stress [MPa]', width=700, height=700, title='Title')          # Plot the data as a line. px.line(df, x, y)

ax.show() 

ax.write_html("Folder/name.html")                       # It saves the plot as an interactive plot. You can also save the plot from the direct output

In [ ]:
### All info
fig = go.Figure()
fig.add_trace(go.Scatter(x=strain1, y=stress1))
fig.add_trace(go.Scatter(x=strain1, y=stress_offset))

fig.add_annotation(x=x, y=y,                                                    ## Plots the maximum stress
            text="Strain={:.4f}, Stress={:.4f}".format(x, y),
            showarrow=True,
            xanchor="right",
            arrowhead=1)
            
fig.add_annotation(x=xi, y=2,                                                   ## Plots the text of the Elastic Modulus
            text=ElasticM,
            showarrow=False,
            arrowhead=1)

fig.add_annotation(x=xi, y=0,                                                   ## Plots de text of the Frature Strength
            text=FractS,
            showarrow=False,
            arrowhead=1)


fig.add_annotation(x=xi, y=yi,                                                  ## Plots the Yield Point
            text="Strain={:.4f}, Stress={:.4f}".format(x1, y1),
            xanchor="right",
            showarrow=True,
            arrowhead=1)

fig.update_layout(showlegend=False, autosize=False, width=700, height=700,      ## Title, and axes font
    title="All info",
    xaxis_title="Strain Ax [-]",
    yaxis_title="Stress [MPa]",
    font=dict(
        family="New Times Roman",
        size=15,
        color="RebeccaPurple"))


fig.update_traces(textposition='top center')
fig.show()

fig.write_html("Folder/name.html")                                              # It saves the plot as an interactive plot. You can also save the plot from the direct output